<!-- title: "Auctioning Used-Car Classifier"
format:
  html:
    code-fold: true
jupyter: python3 -->


#### Author: Andrew Tran


## Blog Post Inspiration and Objectives

In this blog post, I was hoping to look into using Machine Learning to make a prediction system. In particular, I was hoping to imitate ones used for stock prediction. However, since there were too many tutorials on this topic, I decided to investigate into another global, financial factor that governs trade internationally: international currency rates. In the news recently, I have heard rumors of how the international standard of utilizing the US dollar could be compromised with other global powers such as China and India rise in global dominance. Thus, I focused my efforts on analyzing the US currency conversion rates to other countries. With that said, let's try to analyze this topic with some Machine Learning:

## Data Preprocessing - Cleaning and Analytics


In [ ]:
# Import needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from copy import deepcopy
import datetime as dt
plt.style.use("fivethirtyeight")

First, we will read and display the initial dataset in our file system for this blog post, downloaded from Kaggle. This dataset contains loads of valuable information such as almost every major world power's international US currency conversion rate.


In [ ]:
# Reading and displaying the initial dataset
df = pd.read_csv("datasets/foreign_exchange_rates.csv")
df

For clarity on the constraints and parameters of the working datasets, I went to find high-level exploratory statistics on all of the datasets: shape, information about all of the entries, etc.


In [ ]:
# Determining the shape of the initial dataset
df.shape

In [ ]:
# Getting a sample of the initial dataset through the seeing the first 10 entries
# completely in the dataset
df.head()

In [ ]:
# Figuring out all of the columns (and their names) available for me to use in the dataset
df.columns

In [ ]:
# Figuring out the number of duplicated elements in the dataset
# (could be problematic if not resolved)
df.duplicated().sum()

In [ ]:
# Figuring out the number of 'null'/'NaN' elements in the dataset 
# (if NaN filling is needed or not)
(df.isnull().sum() / df.shape[0]) * 100

In [ ]:
# Getting basic information about the dataset
df.info()

Additionally, before handing my combined Book dataset over for Machine Learning training and prediction, I need to clean the data prior to the analysis stage: removing duplicates, deleting null/NaN vales, fixing types of columns, filling invalid values with suitable alternatives, etc.


In [ ]:
# Removing unnecessary/unnamed columns in the dataset
df = df.drop("Unnamed: 0", axis=1)
df

In [ ]:
# Converting the date column into a string representation to a parsable
# Datetime object (needed for later)
df = df.rename(columns={"Time Serie": "DATE"})

def str_to_datetime(date_str: str):
    split = tuple(date_str.split("-"))
    year, month, day = int(split[0]), int(split[1]), int(split[2])
    return dt.datetime(year, month, day)

df["DATE"] = df["DATE"].apply(str_to_datetime)
df["DATE"]

As shown below, I had decided to filter out all of the countries except Austrailia, Canada, the United Kingdom, and Switzerland because those countries had the most stable and consistent international currency rates as well as the most comparable (1:1) currency rates with the United States. One additional reason that these countries were selected for this blog post was do the geographical diversity covered most landmass continents across the world, representative of the global market interactions that the United States deals with on a day-to-day basis.


In [ ]:
# Making the "Date" column the new index (better identifer/key in dataset)
# Taking out other countries except Austrailia, Canada, UK, and 
# Switzerland from dataset for Machine Learning blog post
df.index = df.pop("DATE")
df = df[["AUSTRALIA - AUSTRALIAN DOLLAR/US$", 
         "CANADA - CANADIAN DOLLAR/US$",
         "UNITED KINGDOM - UNITED KINGDOM POUND/US$",
         "SWITZERLAND - FRANC/US$"]]
df

In [ ]:
# Converting of all numerical international currency rates to 2-decimal 
# rates (easier to work with for later)
for col in df:
    df[col] = df[col].map(lambda entry: entry if entry == "ND" else round(float(entry), 2))
df

In [ ]:
# Removing all of the dated rows that contains "ND" as a way to avoid problems 
# when trying to interlay Machine Learning algorithms that require only
# quantitative data with categorical entries
nd_index_list: set[str] = set()
for row_index, _ in df.iterrows():
    for col_entry in df.loc[row_index]:
        if col_entry == "ND":
             nd_index_list.add(row_index)

df = df.drop(labels=list(nd_index_list), axis=0)
df

As shown in the code snippet below, with some online references, the `df_to_windowed_df` function describes a way to use prediction values. A new international currency rate dataframe is created which includes the rows encompassing the 3 previous dates prior to the current date of when the international currency rate was reported as well as the current rate that day. This would be helpful for later Machine Learning predictions because the previous 3 dates and the current will provide it enough context to make more insightful predictions on the future of the international currency rates between that respective country specified and the United States. Later, as you will see, the 3 previous days are going to be the input and the current currency rate (`Target`) will be our output.


In [ ]:
def df_to_windowed_df(data_frame: pd.DataFrame, first_date_str: str, last_date_str: str, n: int, col_name: str):
    first_date = str_to_datetime(first_date_str)
    last_date = str_to_datetime(last_date_str)
    
    target_date = first_date
    
    dates: list = []
    X: list = []
    Y: list = []
    
    last_time = False
    while True:
        df_subset = data_frame.loc[:target_date].tail(n + 1)
        
        if (len(df_subset) != n + 1):
            print("Error: Window of size " + str(n) + " is too large for date " + str(target_date))
            return
        
        values = df_subset[col_name].to_numpy()
        x, y = values[:-1], values[-1]
        
        dates.append(target_date)
        X.append(x)
        Y.append(y)
        
        next_week = data_frame.loc[target_date:target_date + dt.timedelta(days=7)]
        next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
        next_date_str = next_datetime_str.split("T")[0]
        year_month_day = next_date_str.split("-")
        year, month, day = year_month_day
        next_date = dt.datetime(year=int(year), month=int(month), day=int(day))
        
        if last_time:
            break
        
        target_date = next_date
        if (target_date == last_date):
            last_time = True
    
    returned_df = pd.DataFrame({})
    returned_df["Target Date"] = dates
    
    X = np.array(X)
    for i in range(0, n):
        X[:, i]
        returned_df["Target-" + str(n - i)] = X[:, i]
    returned_df["Target"] = Y
    
    return returned_df 

Thus, with the `df_to_windowed_df` function created, we created one for every country to later use for Machine Learning model training and eventual predictions of future rates: Austrailia, Canada, the United Kingdom, and Switzerland.


In [ ]:
# Convert Austrailia's dataframe to a windowed dataframe
windowed_df_austrailia = df_to_windowed_df(data_frame=df, 
                                first_date_str="2000-01-06",
                                last_date_str="2019-12-31",
                                n=3,
                                col_name="AUSTRALIA - AUSTRALIAN DOLLAR/US$")
windowed_df_austrailia  

In [ ]:
# Convert Canada's dataframe to a windowed dataframe
windowed_df_canada = df_to_windowed_df(data_frame=df, 
                                first_date_str="2000-01-06",
                                last_date_str="2019-12-31",
                                n=3,
                                col_name="CANADA - CANADIAN DOLLAR/US$")
windowed_df_canada

In [ ]:
# Convert United Kingdom's dataframe to a windowed dataframe
windowed_df_united_kingdom = df_to_windowed_df(data_frame=df, 
                                first_date_str="2000-01-06",
                                last_date_str="2019-12-31",
                                n=3,
                                col_name="UNITED KINGDOM - UNITED KINGDOM POUND/US$")
windowed_df_united_kingdom

In [ ]:
# Convert Switzerland's dataframe to a windowed dataframe
windowed_df_switzerland = df_to_windowed_df(data_frame=df, 
                                first_date_str="2000-01-06",
                                last_date_str="2019-12-31",
                                n=3,
                                col_name="SWITZERLAND - FRANC/US$")
windowed_df_switzerland

As the Machine Learning model that will be used later in this blog post only accepts `numpy` arrays, we need to extract each of the following quantities from each respective country's dataframe: the dates as an `np.array`, the X-values for input as an 3-dimensional `np.array`, and the Y-values for output as an `np.array`. Thus, with some online references, our codebase now has a function called `windowed_df_to_date_X_y` as shown below to account for this needed transformation.


In [ ]:
def windowed_df_to_date_X_y(windowed_dataframe: pd.DataFrame):
    df_as_np = windowed_dataframe.to_numpy()
    
    dates = df_as_np[:, 0]
    
    middle_matrix = df_as_np[:, 1:-1]
    X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))
    Y = df_as_np[:, -1]
    
    return dates, X.astype(np.float32), Y.astype(np.float32)

dates_austrailia, X_austrailia, y_austrailia = windowed_df_to_date_X_y(windowed_dataframe=windowed_df_austrailia)
dates_canada, X_canada, y_canada = windowed_df_to_date_X_y(windowed_dataframe=windowed_df_canada)
dates_united_kingdom, X_united_kingdom, y_united_kingdom = windowed_df_to_date_X_y(windowed_dataframe=windowed_df_united_kingdom)
dates_switzerland, X_switzerland, y_switzerland = windowed_df_to_date_X_y(windowed_dataframe=windowed_df_switzerland)

print(dates_austrailia.shape, X_austrailia.shape, y_austrailia.shape)
print(dates_canada.shape, X_canada.shape, y_canada.shape)
print(dates_united_kingdom.shape, X_united_kingdom.shape, y_united_kingdom.shape)
print(dates_switzerland.shape, X_switzerland.shape, y_switzerland.shape)
len(windowed_df_austrailia), len(windowed_df_canada), len(windowed_df_united_kingdom), len(windowed_df_switzerland)

Here, I am trying to a visualization of the cleaned dataset before we pass it over for Machine Learning training and prediction. To show the rate of change for the International Currency Rates for Austrailia, Canada, the United Kingdom, and Switzerland over the 20-year period as collected in the data set, I put together a line graph as shown below.


In [ ]:
# Plotting the each four countries currency rates (per US dollar) from 
# 2000 - 2020
plt.plot(df.index, df["AUSTRALIA - AUSTRALIAN DOLLAR/US$"], label="AUS to USD", linewidth=1)
plt.plot(df.index, df[ "CANADA - CANADIAN DOLLAR/US$"], label="CAN to USD", linewidth=1)
plt.plot(df.index, df["UNITED KINGDOM - UNITED KINGDOM POUND/US$"], label="GBR to USD", linewidth=1)
plt.plot(df.index, df["SWITZERLAND - FRANC/US$"], label="CHE to USD", linewidth=1)

plt.legend(loc="upper right")
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date", fontsize=12)
plt.ylabel("Currency Rate of the Respective Country to USD", fontsize=12)
plt.title("Culmination of the All Four Countries' Datasets - After Parsing But Before Machine Learning Alterations")
plt.show()

## Machine Learning - Model Training and Evaluation

Great, now we are onto the Machine Learning part of the blog post!

For this blog post, I worked to create Machine Learning models for each country's dataset separately. I started with Austrailia's data first (as shown below):


In [ ]:
# Getting the location of the 80th and 90th percentile of the number of points 
# in the dataset for later train, vailidation, and test category splitting
percentile_80 = int(len(windowed_df_austrailia) * 0.8)
percentile_90 = int(len(windowed_df_austrailia) * 0.9)
percentile_80, percentile_90

Since the data (date, X, and y) is split into three `np.arrays` and to be more efficient, I will manually split Austrailia's data into train, test, and validation datasets for the Machine Learning model with 80% going to the training dataset, the next 10% going to the validation dataset, and the last 10% going to the test dataset for each `np.array` respectively. This split should be acceptable because I did not mix-and-match training & validation datasets with my testing datasets, wihch would invalidate my entire Machine Learning model. Additionally, looking at the presented in the validation and testing datasets, the range of these respective datasets have been reached before in my training dataset, meaning I anticipated my model should be able to reasonably predict these expected ranges with considerable accuracy. 


In [ ]:
# Splitting Austrailia's data into train, test, and validation sets on 3 
# mediums: the X-axis, the y-axis, and the indices (represented by dates)
dates_austrailia_train, X_austrailia_train, y_austrailia_train = dates_austrailia[:percentile_80], X_austrailia[:percentile_80], y_austrailia[:percentile_80]
dates_austrailia_val, X_austrailia_val, y_austrailia_val = dates_austrailia[percentile_80:percentile_90], X_austrailia[percentile_80:percentile_90], y_austrailia[percentile_80:percentile_90]
dates_austrailia_test, X_austrailia_test, y_austrailia_test = dates_austrailia[percentile_90:], X_austrailia[percentile_90:], y_austrailia[percentile_90:]

plt.plot(dates_austrailia_train, y_austrailia_train, linewidth=1)
plt.plot(dates_austrailia_val, y_austrailia_val, linewidth=1)
plt.plot(dates_austrailia_test, y_austrailia_test, linewidth=1)

plt.legend(["Training", "Validation", "Testing"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Split of Austrailia's Data for LSTM Model")
plt.show()

Now, I began to configure the Machine Learning model. We added Sequential layers: an Input layer 3 by 1 because we will have 3 `np.arrays` of Input and 1 `np.array` as output, utilize a LSTM (Long Short-Term Memory) layer of 64 neurons, apply 2 levels of dense layers with 32 neurons and folliowing recommendations online to use the RELU (Rectified Linear Unit) Activiation Function, and I followed up with one last dense layer of 1 neuron as our output layer since we are just trying to linearly-predict the next currency-rate on a near-future date. Once I configured the Sequential layers, we are ready to compile the model, utilzing the `mean_square_error` as our minimizing loss function, using the `Adam` optimizer, and comparing our trained model against our data with the `mean_absolute_error` metric. Lastly, I fitted our model, utilzing our X_train and Y_train datasets for fitting with validation from our X_valid and Y_valid datasets at 100 epochs.


In [ ]:
# Configuring the Machine Learning Tensorflow Model for Austrailia
austrailia_model = Sequential([layers.Input((3, 1)),
                    layers.LSTM(64),
                    layers.Dense(32, activation="relu"),
                    layers.Dense(32, activation="relu"),
                    layers.Dense(1)])

austrailia_model.compile(loss="mse",
              optimizer=Adam(learning_rate=0.001),
              metrics=["mean_absolute_error"])

austrailia_model.fit(X_austrailia_train, y_austrailia_train, validation_data=(X_austrailia_val, y_austrailia_val), epochs=100)

After the training and fitting of the Machine Learning model for Austrailia, I tried to create visualizations comparing the model against the country's training dataset, validation dataset, but most importantly the testing dataset (as shown below in the line graphs). Note that the darker and thicker blue lines represent the prediction model's projections and the thinner red lines is the observed/gathered data.


In [ ]:
# Testing the Machine Learning Model prediction for Austrailia with the train, 
# validation, and test sets
# Most important is the test set prediction as this tests the effectiveness
# of the Machine Learning model on data it has not seen before 
austrailia_train_pred = austrailia_model.predict(X_austrailia_train).flatten()

plt.plot(dates_austrailia_train, austrailia_train_pred, linewidth=4)
plt.plot(dates_austrailia_train, y_austrailia_train, linewidth=1)
plt.legend(["Training Predictions", "Training Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Austrailia - Training Set Predictions vs. Observations")
plt.show()

austrailia_val_pred = austrailia_model.predict(X_austrailia_val).flatten()

plt.plot(dates_austrailia_val, austrailia_val_pred, linewidth=4)
plt.plot(dates_austrailia_val, y_austrailia_val, linewidth=1)
plt.legend(["Validation Predictions", "Validation Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Austrailia - Validation Set Predictions vs. Observations")
plt.show()

austrailia_test_pred = austrailia_model.predict(X_austrailia_test).flatten()

plt.plot(dates_austrailia_test, austrailia_test_pred, linewidth=4)
plt.plot(dates_austrailia_test, y_austrailia_test, linewidth=1)
plt.legend(["Testing Predictions", "Testing Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Austrailia - Testing Set Predictions vs. Observations")
plt.show()

Through careful consideration of all of the prediction-based vs. observation-based contrast visualizations together, I consolidated all of graphics into one singular visualization for you to see below to get a more general perspective of the effectiveness of the Machine Learning model at training and fitting towards predicting Austrailia's international currency rate with the United States.


In [ ]:
# Plotting Austrailia's observational (reference) data with the predictions of 
# its Machine Learning Model (as a way to visually inspect the effectiveness of 
# the model) 
plt.plot(dates_austrailia_train, austrailia_train_pred, linewidth=4)
plt.plot(dates_austrailia_train, y_austrailia_train, linewidth=1)
plt.plot(dates_austrailia_val, austrailia_val_pred, linewidth=4)
plt.plot(dates_austrailia_val, y_austrailia_val, linewidth=1)
plt.plot(dates_austrailia_test, austrailia_test_pred, linewidth=4)
plt.plot(dates_austrailia_test, y_austrailia_test, linewidth=1)

plt.legend(["Training Predictions",
            "Training Observations",
            "Validation Predictions",
            "Validation Observations",
            "Testing Predictions",
            "Testing Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Culmination of the Three Machine Learning Phases of Austrailia's Dataset - Predictions vs. Observations")
plt.show()

Once Austraila's data was completely trained and visualized, I worked on Canada's data (as shown below):

Since the data (date, X, and y) is split into three `np.arrays` and to be more efficient, I will manually split Canada's data into train, test, and validation datasets for the Machine Learning model with 80% going to the training dataset, the next 10% going to the validation dataset, and the last 10% going to the test dataset for each `np.array` respectively.


In [ ]:
# Splitting Canada's data into train, test, and validation sets on 3 mediums: 
# the X-axis, the y-axis, and the indices (represented by dates)
dates_canada_train, X_canada_train, y_canada_train = dates_canada[:percentile_80], X_canada[:percentile_80], y_canada[:percentile_80]
dates_canada_val, X_canada_val, y_canada_val = dates_canada[percentile_80:percentile_90], X_canada[percentile_80:percentile_90], y_canada[percentile_80:percentile_90]
dates_canada_test, X_canada_test, y_canada_test = dates_canada[percentile_90:], X_canada[percentile_90:], y_canada[percentile_90:]

plt.plot(dates_canada_train, y_canada_train, linewidth=1)
plt.plot(dates_canada_val, y_canada_val, linewidth=1)
plt.plot(dates_canada_test, y_canada_test, linewidth=1)

plt.legend(["Training", "Validation", "Testing"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Split of Canada's Data for LSTM Model")
plt.show()

Now, I began to configure the Machine Learning model. We added Sequential layers: an Input layer 3 by 1 because we will have 3 `np.arrays` of Input and 1 `np.array` as output, utilize a LSTM (Long Short-Term Memory) layer of 64 neurons, apply 2 levels of dense layers with 32 neurons and folliowing recommendations online to use the RELU (Rectified Linear Unit) Activiation Function, and I followed up with one last dense layer of 1 neuron as our output layer since we are just trying to linearly-predict the next currency-rate on a near-future date. Once I configured the Sequential layers, we are ready to compile the model, utilzing the `mean_square_error` as our minimizing loss function, using the `Adam` optimizer, and comparing our trained model against our data with the `mean_absolute_error` metric. Lastly, I fitted our model, utilzing our X_train and Y_train datasets for fitting with validation from our X_valid and Y_valid datasets at 100 epochs.


In [ ]:
# Configuring the Machine Learning Tensorflow Model for Canada
canada_model = Sequential([layers.Input((3, 1)),
                    layers.LSTM(64),
                    layers.Dense(32, activation="relu"),
                    layers.Dense(32, activation="relu"),
                    layers.Dense(1)])

canada_model.compile(loss="mse",
              optimizer=Adam(learning_rate=0.001),
              metrics=["mean_absolute_error"])

canada_model.fit(X_canada_train, y_canada_train, validation_data=(X_canada_val, y_canada_val), epochs=100)

After the training and fitting of the Machine Learning model for Canada, I tried to create visualizations comparing the model against the country's training dataset, validation dataset, but most importantly the testing dataset (as shown below in the line graphs). Note that the darker and thicker blue lines represent the prediction model's projections and the thinner red lines is the observed/gathered data.


In [ ]:
# Testing the Machine Learning Model prediction for Austrailia with the train, 
# validation, and test sets
# Most important is the test set prediction as this tests the effectiveness
# of the Machine Learning model on data it has not seen before 
canada_train_pred = canada_model.predict(X_canada_train).flatten()

plt.plot(dates_canada_train, canada_train_pred, linewidth=4)
plt.plot(dates_canada_train, y_canada_train, linewidth=1)
plt.legend(["Training Predictions", "Training Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Canada - Training Set Predictions vs. Observations")
plt.show()

canada_val_pred = canada_model.predict(X_canada_val).flatten()

plt.plot(dates_canada_val, canada_val_pred, linewidth=4)
plt.plot(dates_canada_val, y_canada_val, linewidth=1)
plt.legend(["Validation Predictions", "Validation Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Canada - Validation Set Predictions vs. Observations")
plt.show()

canada_test_pred = canada_model.predict(X_canada_test).flatten()

plt.plot(dates_canada_test, canada_test_pred, linewidth=4)
plt.plot(dates_canada_test, y_canada_test, linewidth=1)
plt.legend(["Testing Predictions", "Testing Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Canada - Testing Set Predictions vs. Observations")
plt.show()

Through careful consideration of all of the prediction-based vs. observation-based contrast visualizations together, I consolidated all of graphics into one singular visualization for you to see below to get a more general perspective of the effectiveness of the Machine Learning model at training and fitting towards predicting Canada's international currency rate with the United States.


In [ ]:
# Plotting Canada's observational (reference) data with the predictions of its 
# Machine Learning Model (as a way to visually inspect the effectiveness of the 
# model) 
plt.plot(dates_canada_train, canada_train_pred, linewidth=4)
plt.plot(dates_canada_train, y_canada_train, linewidth=1)
plt.plot(dates_canada_val, canada_val_pred, linewidth=4)
plt.plot(dates_canada_val, y_canada_val, linewidth=1)
plt.plot(dates_canada_test, canada_test_pred, linewidth=4)
plt.plot(dates_canada_test, y_canada_test, linewidth=1)

plt.legend(["Training Predictions",
            "Training Observations",
            "Validation Predictions",
            "Validation Observations",
            "Testing Predictions",
            "Testing Observations"], loc="upper right")
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Culmination of the Three Machine Learning Phases of Austrailia's Dataset - Predictions vs. Observations")
plt.show()

Then, I worked on the United Kingdom's data (as shown below):

Since the data (date, X, and y) is split into three `np.arrays` and to be more efficient, I will manually split the United Kingdom's data into train, test, and validation datasets for the Machine Learning model with 80% going to the training dataset, the next 10% going to the validation dataset, and the last 10% going to the test dataset for each `np.array` respectively.


In [ ]:
# Splitting United Kingdom's data into train, test, and validation sets on 3 
# mediums: the X-axis, the y-axis, and the indices (represented by dates)
dates_united_kingdom_train, X_united_kingdom_train, y_united_kingdom_train = dates_united_kingdom[:percentile_80], X_united_kingdom[:percentile_80], y_united_kingdom[:percentile_80]
dates_united_kingdom_val, X_united_kingdom_val, y_united_kingdom_val = dates_united_kingdom[percentile_80:percentile_90], X_united_kingdom[percentile_80:percentile_90], y_united_kingdom[percentile_80:percentile_90]
dates_united_kingdom_test, X_united_kingdom_test, y_united_kingdom_test = dates_united_kingdom[percentile_90:], X_united_kingdom[percentile_90:], y_united_kingdom[percentile_90:]

plt.plot(dates_united_kingdom_train, y_united_kingdom_train, linewidth=1)
plt.plot(dates_united_kingdom_val, y_united_kingdom_val, linewidth=1)
plt.plot(dates_united_kingdom_test, y_united_kingdom_test, linewidth=1)

plt.legend(["Training", "Validation", "Testing"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Split of United Kingdom's Data for LSTM Model")
plt.show()

Now, I began to configure the Machine Learning model. We added Sequential layers: an Input layer 3 by 1 because we will have 3 `np.arrays` of Input and 1 `np.array` as output, utilize a LSTM (Long Short-Term Memory) layer of 64 neurons, apply 2 levels of dense layers with 32 neurons and folliowing recommendations online to use the RELU (Rectified Linear Unit) Activiation Function, and I followed up with one last dense layer of 1 neuron as our output layer since we are just trying to linearly-predict the next currency-rate on a near-future date. Once I configured the Sequential layers, we are ready to compile the model, utilzing the `mean_square_error` as our minimizing loss function, using the `Adam` optimizer, and comparing our trained model against our data with the `mean_absolute_error` metric. Lastly, I fitted our model, utilzing our X_train and Y_train datasets for fitting with validation from our X_valid and Y_valid datasets at 100 epochs.


In [ ]:
# Configuring the Machine Learning Tensorflow Model for United Kingdom
united_kingdom_model = Sequential([layers.Input((3, 1)),
                    layers.LSTM(64),
                    layers.Dense(32, activation="relu"),
                    layers.Dense(32, activation="relu"),
                    layers.Dense(1)])

united_kingdom_model.compile(loss="mse",
              optimizer=Adam(learning_rate=0.001),
              metrics=["mean_absolute_error"])

united_kingdom_model.fit(X_united_kingdom_train, y_united_kingdom_train, validation_data=(X_united_kingdom_val, y_united_kingdom_val), epochs=100)

After the training and fitting of the Machine Learning model for the United Kingdom, I tried to create visualizations comparing the model against the country's training dataset, validation dataset, but most importantly the testing dataset (as shown below in the line graphs). Note that the darker and thicker blue lines represent the prediction model's projections and the thinner red lines is the observed/gathered data.


In [ ]:
# Testing the Machine Learning Model prediction for United Kingdom with the train, 
# validation, and test sets
# Most important is the test set prediction as this tests the effectiveness
# of the Machine Learning model on data it has not seen before 
united_kingdom_train_pred = united_kingdom_model.predict(X_united_kingdom_train).flatten()

plt.plot(dates_united_kingdom_train, united_kingdom_train_pred, linewidth=4)
plt.plot(dates_united_kingdom_train, y_united_kingdom_train, linewidth=1)
plt.legend(["Training Predictions", "Training Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("United Kingdom - Training Set Predictions vs. Observations")
plt.show()

united_kingdom_val_pred = united_kingdom_model.predict(X_united_kingdom_val).flatten()

plt.plot(dates_united_kingdom_val, united_kingdom_val_pred, linewidth=4)
plt.plot(dates_united_kingdom_val, y_united_kingdom_val, linewidth=1)
plt.legend(["Validation Predictions", "Validation Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("United Kingdom - Validation Set Predictions vs. Observations")
plt.show()

united_kingdom_test_pred = united_kingdom_model.predict(X_united_kingdom_test).flatten()

plt.plot(dates_united_kingdom_test, united_kingdom_test_pred, linewidth=4)
plt.plot(dates_united_kingdom_test, y_united_kingdom_test, linewidth=1)
plt.legend(["Testing Predictions", "Testing Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("United Kingdom - Testing Set Predictions vs. Observations")
plt.show()

Through careful consideration of all of the prediction-based vs. observation-based contrast visualizations together, I consolidated all of graphics into one singular visualization for you to see below to get a more general perspective of the effectiveness of the Machine Learning model at training and fitting towards predicting the United Kingdom's international currency rate with the United States.


In [ ]:
# Plotting United Kingdom's observational (reference) data with the predictions 
# of its Machine Learning Model (as a way to visually inspect the effectiveness 
# of the model) 
plt.plot(dates_united_kingdom_train, united_kingdom_train_pred, linewidth=4)
plt.plot(dates_united_kingdom_train, y_united_kingdom_train, linewidth=1)
plt.plot(dates_united_kingdom_val, united_kingdom_val_pred, linewidth=4)
plt.plot(dates_united_kingdom_val, y_united_kingdom_val, linewidth=1)
plt.plot(dates_united_kingdom_test, united_kingdom_test_pred, linewidth=4)
plt.plot(dates_united_kingdom_test, y_united_kingdom_test, linewidth=1)

plt.legend(["Training Predictions",
            "Training Observations",
            "Validation Predictions",
            "Validation Observations",
            "Testing Predictions",
            "Testing Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Culmination of the Three Machine Learning Phases of United Kingdom's Dataset - Predictions vs. Observations")
plt.show()

Lastly, I worked on Switzerland's data (as shown below):

Since the data (date, X, and y) is split into three `np.arrays` and to be more efficient, I will manually split Switzerland's data into train, test, and validation datasets for the Machine Learning model with 80% going to the training dataset, the next 10% going to the validation dataset, and the last 10% going to the test dataset for each `np.array` respectively.


In [ ]:
# Splitting Switzerland's data into train, test, and validation sets on 3 
# mediums: the X-axis, the y-axis, and the indices (represented by dates)
dates_switzerland_train, X_switzerland_train, y_switzerland_train = dates_switzerland[:percentile_80], X_switzerland[:percentile_80], y_switzerland[:percentile_80]
dates_switzerland_val, X_switzerland_val, y_switzerland_val = dates_switzerland[percentile_80:percentile_90], X_switzerland[percentile_80:percentile_90], y_switzerland[percentile_80:percentile_90]
dates_switzerland_test, X_switzerland_test, y_switzerland_test = dates_switzerland[percentile_90:], X_switzerland[percentile_90:], y_switzerland[percentile_90:]

plt.plot(dates_switzerland_train, y_switzerland_train, linewidth=1)
plt.plot(dates_switzerland_val, y_switzerland_val, linewidth=1)
plt.plot(dates_switzerland_test, y_switzerland_test, linewidth=1)

plt.legend(["Training", "Validation", "Testing"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Split of Switzerland's Data for LSTM Model")
plt.show()

Now, I began to configure the Machine Learning model. We added Sequential layers: an Input layer 3 by 1 because we will have 3 `np.arrays` of Input and 1 `np.array` as output, utilize a LSTM (Long Short-Term Memory) layer of 64 neurons, apply 2 levels of dense layers with 32 neurons and folliowing recommendations online to use the RELU (Rectified Linear Unit) Activiation Function, and I followed up with one last dense layer of 1 neuron as our output layer since we are just trying to linearly-predict the next currency-rate on a near-future date. Once I configured the Sequential layers, we are ready to compile the model, utilzing the `mean_square_error` as our minimizing loss function, using the `Adam` optimizer, and comparing our trained model against our data with the `mean_absolute_error` metric. Lastly, I fitted our model, utilzing our X_train and Y_train datasets for fitting with validation from our X_valid and Y_valid datasets at 100 epochs.


In [ ]:
# Configuring the Machine Learning Tensorflow Model for Switzerland
switzerland_model = Sequential([layers.Input((3, 1)),
                    layers.LSTM(64),
                    layers.Dense(32, activation="relu"),
                    layers.Dense(32, activation="relu"),
                    layers.Dense(1)])

switzerland_model.compile(loss="mse",
              optimizer=Adam(learning_rate=0.001),
              metrics=["mean_absolute_error"])

switzerland_model.fit(X_switzerland_train, y_switzerland_train, validation_data=(X_switzerland_val, y_switzerland_val), epochs=100)

After the training and fitting of the Machine Learning model for Switzerland, I tried to create visualizations comparing the model against the country's training dataset, validation dataset, but most importantly the testing dataset (as shown below in the line graphs). Note that the darker and thicker blue lines represent the prediction model's projections and the thinner red lines is the observed/gathered data.


In [ ]:
# Testing the Machine Learning Model prediction for Switzerland with the train, 
# validation, and test sets
# Most important is the test set prediction as this tests the effectiveness
# of the Machine Learning model on data it has not seen before 
switzerland_train_pred = switzerland_model.predict(X_switzerland_train).flatten()

plt.plot(dates_switzerland_train, switzerland_train_pred, linewidth=4)
plt.plot(dates_switzerland_train, y_switzerland_train, linewidth=1)
plt.legend(["Training Predictions", "Training Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Switzerland - Training Set Predictions vs. Observations")
plt.show()

switzerland_val_pred = switzerland_model.predict(X_switzerland_val).flatten()

plt.plot(dates_switzerland_val, switzerland_val_pred, linewidth=4)
plt.plot(dates_switzerland_val, y_switzerland_val, linewidth=1)
plt.legend(["Validation Predictions", "Validation Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Switzerland - Validation Set Predictions vs. Observations")
plt.show()

switzerland_test_pred = switzerland_model.predict(X_switzerland_test).flatten()

plt.plot(dates_switzerland_test, switzerland_test_pred, linewidth=4)
plt.plot(dates_switzerland_test, y_switzerland_test, linewidth=1)
plt.legend(["Testing Predictions", "Testing Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Switzerland - Testing Set Predictions vs. Observations")
plt.show()

Through careful consideration of all of the prediction-based vs. observation-based contrast visualizations together, I consolidated all of graphics into one singular visualization for you to see below to get a more general perspective of the effectiveness of the Machine Learning model at training and fitting towards predicting Switzerland's international currency rate with the United States.


In [ ]:
# Plotting Switzerland's observational (reference) data with the predictions 
# of its Machine Learning Model (as a way to visually inspect the effectiveness 
# of the model) 
plt.plot(dates_switzerland_train, switzerland_train_pred, linewidth=4)
plt.plot(dates_switzerland_train, y_switzerland_train, linewidth=1)
plt.plot(dates_switzerland_val, switzerland_val_pred, linewidth=4)
plt.plot(dates_switzerland_val, y_switzerland_val, linewidth=1)
plt.plot(dates_switzerland_test, switzerland_test_pred, linewidth=4)
plt.plot(dates_switzerland_test, y_switzerland_test, linewidth=1)

plt.legend(["Training Predictions",
            "Training Observations",
            "Validation Predictions",
            "Validation Observations",
            "Testing Predictions",
            "Testing Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Culmination of the Three Machine Learning Phases of Switzerland's Dataset - Predictions vs. Observations")
plt.show()

To put it all together, I made one large visualization to encompass all 4 countries' Machine Learning model prediction-based vs. observation-based visualizations - Austrailia, Canada, the United Kingdom, and Switzerland as shown below.


In [ ]:
# Plotting all countries' observational (reference) data with the predictions 
# of its Machine Learning Model (as a way to visually inspect the effectiveness 
# of the model in an overall sense) 
plt.plot(dates_austrailia_train, austrailia_train_pred, linewidth=4, linestyle="solid")
plt.plot(dates_austrailia_train, y_austrailia_train, linewidth=1, linestyle="solid")
plt.plot(dates_austrailia_val, austrailia_val_pred, linewidth=4, linestyle="solid")
plt.plot(dates_austrailia_val, y_austrailia_val, linewidth=1, linestyle="solid")
plt.plot(dates_austrailia_test, austrailia_test_pred, linewidth=4, linestyle="solid")
plt.plot(dates_austrailia_test, y_austrailia_test, linewidth=1, linestyle="solid")

plt.plot(dates_canada_train, canada_train_pred, linewidth=4, linestyle="dashed")
plt.plot(dates_canada_train, y_canada_train, linewidth=1, linestyle="dashed")
plt.plot(dates_canada_val, canada_val_pred, linewidth=4, linestyle="dashed")
plt.plot(dates_canada_val, y_canada_val, linewidth=1, linestyle="dashed")
plt.plot(dates_canada_test, canada_test_pred, linewidth=4, linestyle="dashed")
plt.plot(dates_canada_test, y_canada_test, linewidth=1, linestyle="dashed")

plt.plot(dates_united_kingdom_train, united_kingdom_train_pred, linewidth=4, linestyle="dotted")
plt.plot(dates_united_kingdom_train, y_united_kingdom_train, linewidth=1, linestyle="dotted")
plt.plot(dates_united_kingdom_val, united_kingdom_val_pred, linewidth=4, linestyle="dotted")
plt.plot(dates_united_kingdom_val, y_united_kingdom_val, linewidth=1, linestyle="dotted")
plt.plot(dates_united_kingdom_test, united_kingdom_test_pred, linewidth=4, linestyle="dotted")
plt.plot(dates_united_kingdom_test, y_united_kingdom_test, linewidth=1, linestyle="dotted")

plt.plot(dates_switzerland_train, switzerland_train_pred, linewidth=4, linestyle="dashdot")
plt.plot(dates_switzerland_train, y_switzerland_train, linewidth=1, linestyle="dashdot")
plt.plot(dates_switzerland_val, switzerland_val_pred, linewidth=4, linestyle="dashdot")
plt.plot(dates_switzerland_val, y_switzerland_val, linewidth=1, linestyle="dashdot")
plt.plot(dates_switzerland_test, switzerland_test_pred, linewidth=4, linestyle="dashdot")
plt.plot(dates_switzerland_test, y_switzerland_test, linewidth=1, linestyle="dashdot")

plt.legend(["Training Predictions",
            "Training Observations",
            "Validation Predictions",
            "Validation Observations",
            "Testing Predictions",
            "Testing Observations"])
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Culmination of the Three Machine Learning Phases of All Four Countries' Datasets - Predictions vs. Observations")
plt.show()

Ultimately, here is the moment of truth: I tried to recursively predict Austrailia's future expected projection on its international currency rate between the United States through using my LSTM prediction model as shown below. By using the data from the training dataset for Austrailia, I tried to contrast the effectiveness of the model by constrasting its projection to the actual validation and testing data results.
However, unfortunately, it did not predict as well as I hoped, but this was a good experiment, nonetheless.


In [ ]:
# Attempt at using the model to predict into the future for Austrailia's currency 
# rate
recursive_pred: list = []
recursive_dates = np.concatenate([dates_austrailia_val, dates_austrailia_test])

for target_date in recursive_dates:
    last_window = deepcopy(X_austrailia_train[-1])
    next_pred = austrailia_model.predict(np.array([last_window])).flatten()
    recursive_pred.append(next_pred)
    last_window[-1] = next_pred

To visualize the results of my recursive prediction that I completed, I added these results to the previous large visualization that encompassed all 4 countries' Machine Learning model prediction-based vs. observation-based visualizations - Austrailia, Canada, the United Kingdom, and Switzerland as shown below.


In [ ]:
# Plotting all countries' observational (reference) data with the predictions 
# of its Machine Learning Model (as a way to visually inspect the effectiveness 
# of the model in an overall sense) along with the new recursive prediction
# results
plt.plot(dates_austrailia_train, austrailia_train_pred, linewidth=4, linestyle="solid")
plt.plot(dates_austrailia_train, y_austrailia_train, linewidth=1, linestyle="solid")
plt.plot(dates_austrailia_val, austrailia_val_pred, linewidth=4, linestyle="solid")
plt.plot(dates_austrailia_val, y_austrailia_val, linewidth=1, linestyle="solid")
plt.plot(dates_austrailia_test, austrailia_test_pred, linewidth=4, linestyle="solid")
plt.plot(dates_austrailia_test, y_austrailia_test, linewidth=1, linestyle="solid")
plt.plot(recursive_dates, recursive_pred, linewidth=4, linestyle="solid")

plt.legend(["Training Predictions",
            "Training Observations",
            "Validation Predictions",
            "Validation Observations",
            "Testing Predictions",
            "Testing Observations",
            "Recursive Predictions"], loc="upper right")
plt.rcParams["font.size"] = 8
plt.tight_layout()
plt.xlabel("Historical Date")
plt.ylabel("Currency Rate of the Respective Country to USD")
plt.title("Culmination of the Three Machine Learning Phases of Austrailia's Dataset - Predictions vs. Observations (With Recursive Pred. Attempt)")
plt.show()

## Conclusions

- Returning to my attempt to test the ability for my Machine Learning model to predict the International Currency rates for Austrailia from just the range of my training dataset as shown above, I felt that its prediction was acceptable in my opinion giving the averaging across the currency rate values in my validation and testing datasets would have proven to be considerably close in its precision. 

- Thus, I reached the conclusion that my International Currency Predictor would not be a viable Machine Learning model for any real-life applications. This Machine Learning model raised a valuable point - financial market prediction is a difficult endeavor to accurately predict. From a more general point of view, this International Currency Maching Learning prediction model is not totally inaccurate or untrustworthy but rather it is only on the micro-scale. These financial predictions do not always account for current events in that respective country such as changing domestic or international relations or constantly changing socioeconomic trends which could significantly sway market confidence and impact currency rate fluctuation. Consequently, economists often have to look at larger economic trends across several decades in order to make more educated predictions. 

- Ultimately, I learned a great deal about the complexities of financial prediction and usage of Machine Learning models as an attempt to solve this issue. Even though my model was not as successful at predicting as I would have hoped, I am interested to see the future of Machine Learning as it soon dominates these economic sub-field and  continues to rise in its relevance by being a pivotal talking point for the future and applied soon enough to other sectors of our society. 

## Reference Sources and Citations (IEEE Format)

To complete this blog post, I used the following online sources as references for developing this:

[1] US International Exchange Rates Dataset:

B. Ferreira, "Foreign Exchange Rates 2000-2019", 2019. [Online]. Available: https://www.kaggle.com/datasets/brunotly/foreign-exchange-rates-per-dollar-20002019. [Accessed: 04-Sep.-2023].

[2] Tutorial on Basics of LSTM and Prediction & Forecasting in Machine Learning:

Greg Hogg, "Stock Price Prediction & Forecasting with LSTM Neural Networks in Python", *YouTube*, 26-Mar.-2022. [Online.] Available: https://www.youtube.com/watch?v=CbTU92pbDKw. [Accessed: 06-Sep.-2023]. 
